Aim: develop a turnkey quicklook code for X-SAPR2

In [1]:
#all our favourite imports
from matplotlib import pyplot as plt
import numpy as np
import pyart
from netCDF4 import num2date
import pytz
import cartopy
import os
#warnings.filterwarnings("ignore")
%matplotlib inline


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/Users/scollis/anaconda/envs/houston/lib/python3.5/json/encoder.py:198: DeprecationWarning: Interpreting naive datetime as local 2017-07-11 10:33:40.180835. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [2]:
filename = '/data/azores_pyart/enaxsaprsecD1.00.20170625.212006.raw.sec_XSAPR2_20170625212006_00.h5'
ena_radar =  pyart.aux_io.gamic_hdf5.read_gamic(filename, file_field_names=True)


/Users/scollis/anaconda/envs/houston/lib/python3.5/json/encoder.py:198: DeprecationWarning: Interpreting naive datetime as local 2017-07-11 10:35:10.193011. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [3]:
quicklook_directory = '/data/azores_pyart/quicklooks/'

dict_keys(['data', '_FillValue', 'units'])


/Users/scollis/anaconda/envs/houston/lib/python3.5/json/encoder.py:198: DeprecationWarning: Interpreting naive datetime as local 2017-07-11 10:36:03.496885. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [7]:
for field_name in list(ena_radar.fields.keys()):
    try:
        print(field_name, ':', ena_radar.fields[field_name]['units'])
    except KeyError:
        print(field_name, ':', 'Unavailable')

Z : dBZ
UZ : dBZ
UZDR1 : dB
ZDR1 : dB
CWv : m/s
AZh : dBZ
Wh : m/s
UnVh : m/s
SNRv : dB
UPHIDP : deg
KDP : deg/km
AZDR : dB
RHOHV : Unavailable
ZDR : dB
Wv : m/s
Vv : m/s
UZv : dBZ
SQIh : Unavailable
PHIDP : deg
CMAP : Unavailable
SNRh : dB
Vh : m/s
CWh : m/s
AZDR1 : dB
UZh : dBZ
Zv : dBZ
URHOHV : Unavailable
Zh : dBZ
CLASS : Unavailable
UZDR : dB
UnVv : m/s
SQIv : Unavailable


/Users/scollis/anaconda/envs/houston/lib/python3.5/json/encoder.py:198: DeprecationWarning: Interpreting naive datetime as local 2017-07-11 11:10:40.722629. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [8]:
#page 44 of https://github.com/scollis/CfRadial/blob/master/docs/CfRadialDoc.v2.0.draft.pdf
z_name = 'equivalent_reflectivity_factor'
v_name = 'radial_velocity_of_scatterers_away_from_instrument'
wth_name = 'doppler_spectrum_width'
zdr_name = 'log_differential_reflectivity_hv'
ldr_name = 'log_linear_depolarization_ratio_hv'
phidp_name = 'differential_phase_hv'
kdp_name = 'specific_differential_phase_hv'
rhv_name = 'cross_correlation_ratio_hv' 
power_name = 'log_power'
sqi_name = 'normalized_coherent_power'
zc_name = 'corrected_equivalent_reflectivity_factor'
vc_name = 'corrected_radial_velocity_of_scatterers_away_from_instrument'
zdrc_name = 'corrected_log_differential_reflectivity_hv'
class_name = 'radar_echo_classification'
snr_name = 'signal_to_noise_ratio'

trans_table = {'Z': {'standard_name': z_name, 'name': 'refelctivity'},
               'UZ': {'standard_name': z_name, 'name': 'uncorrected_refelctivity'},
              'UZDR1': {'standard_name': zdr_name, 'name': 'uncorrected_differential_reflectivity_1'},
              'ZDR1': {'standard_name': zdr_name, 'name': 'differential_refelctivity_1'},
              'CWv': {'standard_name': wth_name, 'name': 'corrected_spectral_width_vertical'},
              'AZh': {'standard_name': z_name, 'name': 'attenuation_corrected_reflectivity_horizontal'},
              'Wh': {'standard_name': wth_name, 'name': 'spectral_width_horizontal'},
              'UnVh': {'standard_name': vc_name, 'name': 'unfolded_radial_velocity'},
              'SNRv': {'standard_name': snr_name, 'name': 'signal_to_noise_ratio_vertical'},
              'UPHIDP': {'standard_name': phidp_name, 'name': 'unfolded_differential_phase'},
              'KDP': {'standard_name': kdp_name, 'name': 'specific_differential_phase'},
              'AZDR': {'standard_name': zdrc_name, 'name': 'attenuation_corrected_differential_reflectivity'},
              'RHOHV': {'standard_name': rhv_name, 'name': 'cross_correlation_ratio_hv', 'units': 'unitless'},
              'ZDR': {'standard_name': zdr_name, 'name': 'differential_refelctivity'},
              'Wv': {'standard_name': wth_name, 'name': 'spectral_width_vertical'},
              'Vv': {'standard_name': v_name, 'name': 'radial_velocity_vertical'},
              'UZv': {'standard_name': z_name, 'name': 'uncorrected_reflectivity_vertical'},
              'SQIh': {'standard_name': sqi_name, 'name': 'normalized_coherent_power_horizontal', 'units': 'unitless'},
              'PHIDP': {'standard_name': phidp_name, 'name': 'differential_phase'},
              'CMAP': {'standard_name': class_name, 'name': 'clutter_map'},
              'SNRh': {'standard_name': snr_name, 'name': 'signal_to_noise_ratio_horizontal'},
              'Vh': {'standard_name': v_name, 'name': 'radial_velocity_horizontal'},
              'CWh': {'standard_name': wth_name, 'name': 'corrected_spectral_width_horizontal'},
              'AZDR1': {'standard_name': zdrc_name, 'name': 'attenuation_corrected_differential_reflectivity_1'},
              'UZh': {'standard_name': z_name, 'name': 'uncorrected_refelctivity_horizontal'},
              'Zv': {'standard_name': z_name, 'name': 'refelctivity_horizontal'}}

/Users/scollis/anaconda/envs/houston/lib/python3.5/json/encoder.py:198: DeprecationWarning: Interpreting naive datetime as local 2017-07-11 11:46:58.116062. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


linear_equivalent_reflectivity_factor Z Z no
log_linear_depolarization_ratio_hv LDR dB no
log_linear_depolarization_ratio_h LDRH dB no
log_linear_depolarization_ratio_v LDRV dB no


cross_polar_differential_phase PHIHX degrees no
cross_correlation_ratio_hv RHOHV no
co_to_cross_polar_correlation_ratio_h RHOHX no
co_to_cross_polar_correlation_ratio_v RHOXV no
log_power DBM dBm no
log_power_co_polar_h DBMHC dBm no
log_power_cross_polar_h DBMHX dBm no
log_power_co_polar_v DBMVC dBm no
log_power_cross_polar_v DBMVX dBm no
linear_power PWR mW no
linear_power_co_polar_h PWRHC mW no
linear_power_cross_polar_h PWRHX mW no
linear_power_co_polar_v PWRVC mW no
CfRadial format Version 2.0 DRAFT 2016-08-12
EOL, NCAR 45
Standard name Short
name
Units Already
in CF?
linear_power_cross_polar_v PWRVX mW no
signal_to_noise_ratio SNR dB no
signal_to_noise_ratio_co_polar_h SNRHC dB no
signal_to_noise_ratio_cross_polar_h SNRHX dB no
signal_to_noise_ratio_co_polar_v SNRVC dB no
signal_to_noise_ratio_cross_polar_v SNRVX dB no



radar_estimated_rain_rate RRR mm/hr no
rain_rate RR kg/m2/s yes
radar_echo_classification
(should be used for PID, HCA, HID etc)
REC legend no